> Given an array of integers, return a new array such that each element at index `i` of the new array is the product of all the numbers in the original array except the one at `i`.

>For example, if our input was `[1, 2, 3, 4, 5]`, the expected output would be `[120, 60, 40, 30, 24]`. If our input was `[3, 2, 1]`, the expected output would be `[2, 3, 6]`.

# Basic
First, here is a simple solution.
Time complexity : `O(N + N)`.

In [1]:
from math import prod

num_lists = [
    [1, 2, 3, 4, 5],
    [3, 2, 1],
]

def product_list(num_list: list) -> list:
    final_array = []
    for i, n in enumerate(num_list):
        if i == 0:
            final_array.append(prod(num_list[1:]))
        elif i == len(num_list) - 1:
            final_array.append(prod(num_list[:-1]))
        else:
            final_array.append(prod(num_list[:i] + num_list[i + 1:]))
    return final_array

for num_list in num_lists:
    print(num_list, "=>", product_list(num_list))

[1, 2, 3, 4, 5] => [120, 60, 40, 30, 24]
[3, 2, 1] => [2, 3, 6]


Ok, so we got something working ; let's see if we can improve the performance.

First off, I'm thinking of something like a [Merkle Tree](https://en.wikipedia.org/wiki/Merkle_tree)-style Binary Tree, with each node [memoized](https://en.wikipedia.org/wiki/Memoization).

Let's try.

In [ ]:
...

## Update : a few hours later
Hm... Yeah...

I kind of struggled to find a way of implementing such a tree without it being more computationally expensive than the basic solution.

Here's what I came up to :

In [2]:
def product_list_improved(num_list: list) -> list:

    def process_couple(num_list: list, i: int, memo: dict) -> int:
        result = 1
        for x in range(len(num_list)):
            # For the next loop, we process the values as couples (x, x+1)
            if x % 2 == 0:
                if i is x:
                    try:
                        result *= num_list[x + 1]
                    except IndexError:
                        # This case only happens when i and x are at
                        # the last value in the list ; 
                        # therefore we can return the result.
                        return result
                elif i is (x + 1):
                    result *= num_list[x]
                else:
                    try:
                        result *= memo[str(x) + str(x + 1)] # May raise KeyError (unlikely)
                    except KeyError:
                        result *= memo[str(x) + "1"]
        return result

    final_array = []
    memo = {}

    # Construct 2D tree in variable "memo"
    for i in range(0, len(num_list), 2):
        name = str(i)
        value = num_list[i]
        try:
            # Try to add the value to i, so that we have a couple.
            value *= num_list[i + 1]
            name += str(i + 1)
        except IndexError:
            name += "1"
        memo[name] = value
    
    for i in range(len(num_list)):
        final_array.append(process_couple(num_list, i, memo))

    return final_array

for num_list in num_lists:
    print(num_list, "=>", product_list_improved(num_list))

[1, 2, 3, 4, 5] => [120, 60, 40, 30, 24]
[3, 2, 1] => [2, 3, 6]


**Neat !** And much more compact than what I though I could do !

As you can see, the tree is 2D.
One could try to make it 3D, not me, I've already spent way too much time on this problem.

I'd say time complexity is `O(N + N / 2)` and space complexity is `O(N / 2)`.
I'm not 100% sure these are the right values, so please correct me if I'm wrong !

>I couldn't find a way to construct the memo along the way, so this function builds it beforehand, that's why the complexity is very similar to the basic example.

Now, let's experiment !

In [3]:
# First, let's do verifications

from random import randint

for i in [2, 3, 4, 5]:
    new_list = [randint(1, i * 10) for _ in range(i)]
    results1 = product_list(new_list)
    results2 = product_list_improved(new_list)
    if results1 == results2:
        print(f"GOOD: Results are the same for both functions !")
    else:
        print(f"BAD: Results are NOT the same !")
    print(new_list, results1, results2)

GOOD: Results are the same for both functions !
[10, 15] [15, 10] [15, 10]
GOOD: Results are the same for both functions !
[26, 10, 23] [230, 598, 260] [230, 598, 260]
GOOD: Results are the same for both functions !
[2, 2, 20, 26] [1040, 1040, 104, 80] [1040, 1040, 104, 80]
GOOD: Results are the same for both functions !
[37, 12, 40, 5, 14] [33600, 103600, 31080, 248640, 88800] [33600, 103600, 31080, 248640, 88800]


In [4]:
# Second test: speeeeed

import time

for i in [10, 20, 30, 40, 50, 100, 250]:
    for fc in ["product_list", "product_list_improved"]:
        # We're making the difficulty exponential
        new_list = [int("1" * i) for x in range(i)]
        tic = time.time()
        results = globals()[fc](new_list)
        toc = time.time()
        print(f"It took {toc - tic} seconds to process a list of {i} integers",
              f"with function {fc}.")
    print("\n")

It took 0.0 seconds to process a list of 10 integers with function product_list.
It took 0.0 seconds to process a list of 10 integers with function product_list_improved.


It took 0.0 seconds to process a list of 20 integers with function product_list.
It took 0.001001119613647461 seconds to process a list of 20 integers with function product_list_improved.


It took 0.0 seconds to process a list of 30 integers with function product_list.
It took 0.0009996891021728516 seconds to process a list of 30 integers with function product_list_improved.


It took 0.0029997825622558594 seconds to process a list of 40 integers with function product_list.
It took 0.0020017623901367188 seconds to process a list of 40 integers with function product_list_improved.


It took 0.005999326705932617 seconds to process a list of 50 integers with function product_list.
It took 0.007002115249633789 seconds to process a list of 50 integers with function product_list_improved.


It took 0.19299769401550293 se

Meh...

I'd say it is not that fast because it's Python. Migrated over a performance-focused language such as C++, the stats would be very different I believe.

Do whatever want with that :)